# Generating UI (data visualizations) for index.html

In [1]:
import sqlite3
import pandas as pd
import json


# connect to database

conn = sqlite3.connect("Submission.db")

Idea: Create an interactive bar graph matrix

Based on the attributes, we create a bar graph matrix comparing injury vs non injury based on weather

Each bar graph is then clickable, which opens a secondary stratified line graph (by court type)

In [ ]:
# peek datadbase

cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

# one table called Data

# read column names
cursor.execute("PRAGMA table_info(Data)")
print(cursor.fetchall())

# print first row
cursor.execute("SELECT * FROM Data LIMIT 1")
print(cursor.fetchall())



[('Data',)]
[(0, 'Date', 'TEXT', 0, None, 0), (1, 'Player', 'TEXT', 0, None, 0), (2, 'Tournament', 'TEXT', 0, None, 0), (3, 'Location', 'TEXT', 0, None, 0), (4, 'Surface', 'TEXT', 0, None, 0), (5, 'Injury', 'INTEGER', 0, None, 0), (6, 'Weather', 'TEXT', 0, None, 0)]
[('2000-01-03', 'Dosedel S.', 'Australian Hardcourt Championships', 'Adelaide', 'Hard', 0, '{"date": "2000-01-03", "weather_code": [2.0], "temperature_2m_max": [25.19649887084961], "temperature_2m_min": [12.796499252319336], "temperature_2m_mean": [18.254831314086914], "daylight_duration": [51965.80859375], "precipitation_sum": [0.0], "rain_sum": [0.0], "snowfall_sum": [0.0], "precipitation_hours": [0.0], "wind_speed_10m_max": [31.259944915771484], "wind_gusts_10m_max": [47.880001068115234], "wind_direction_10m_dominant": [195.9699249267578]}')]


Findings:

1. 0 means not injured, 1 means injured

2. Weather is composed of:

date (YYYY-MM-DD)

weather_code following the pattern:

* 0 = Clear sky
* 1 = Mainly clear
* 2 = Partly cloudy
* 3 = Overcast
* 45 = Fog
* 48 = Depositing Rime Fog
* 51 = Drizzle: Light
* 53 = Drizzle: Moderate
etc .... 

temperature_2m_max (maximum temperature at 2 meters above the ground for the day)

temperature_2m_min
(minimum temperature at 2 meters above the ground for the day)

temperature_2m_mean
(mean temperature at 2 meters above the ground for the day)

daylight_duration
(duration of daylight for the day in seconds)

precipitation_sum
(total amount of precipitation for the day (in milimeters))

rain_sum
(total amount of rain during the day (in milimeters))

snowfall_sum
(total amount of snowfall during the day (in milimeters))

precipitation_hours 
(number of hours during which precipitation was recorded)

wind_speed_10m_max
(maximum wind speed at 10 meters above the ground for the day)

wind_gusts_10m_max
(maximum wind gust at 10 meters above the ground for the day)

wind_direction_10m_dominant
(dominant wind direction at 10 meters above the ground for the day, in degrees)



In [41]:
# print weather data
cursor.execute("SELECT DISTINCT json_extract(Weather, '$.weather_code') FROM Data")
print(cursor.fetchall())

# get all players

cursor.execute("SELECT DISTINCT Player FROM Data")
print(cursor.fetchall())


[('[2.0]',), ('[1.0]',), ('[51.0]',), ('[61.0]',), (None,)]
[('Dosedel S.',), ('Clement A.',), ('Escude N.',), ('Knippschild J.',), ('Fromberg R.',), ('Arthurs W.',), ('Grosjean S.',), ('Balcells J.',), ('Hewitt L.',), ('Tebbutt M.',), ('Martin A.',), ('Draper S.',), ('Petrovic D.',), ('Pozzi G.',), ('Stoltenberg J.',), ('Goldstein P.',), ('Enqvist T.',), ('Lisnard J.',), ('Sekulov J.',), ('Norman M.',), ('Vinck C.',), ('Henman T.',), ('Kumar S.',), ('Behrend T.',), ('Rochus C.',), ('Delgado J.',), ('Spottl M.',), ('Hantschk M.',), ('Kroslak J.',), ('Ketola T.',), ('Zib T.',), ('Paes L.',), ('Ram A.',), ('Sanguinetti D.',), ('Kilderry P.',), ('Tabara M.',), ('Ishii Y.',), ('Vinciguerra A.',), ('Agenor R.',), ('Jonsson F.',), ('Vanek J.',), ('Manta L.',), ('Golmard J.',), ('Pioline C.',), ('Damm M.',), ('Bastl G.',), ('Al-Alawi S.K.',), ('Caratti C.',), ('Arazi H.',), ('El Aynaoui Y.',), ('Luxa P.',), ('Kiefer N.',), ('Puerta M.',), ('Santoro F.',), ('Diaz J.',), ('Schuettler R.',), ('S

Observations:

Weather is overall restricted to the following conditions (see https://open-meteo.com/en/docs#daily=weather_code):

2 = Partly cloudy
1 = Mainly clear
51 = Light drizzle
61 = Slight rain 

In [39]:
def generic_bar(): # generic bar graph (injuries per court type - 3 layers)
    # Filter only those results from July (07) 2022 to February (02) 2023
    query = "SELECT Surface, Injury FROM Data WHERE Date BETWEEN '2022-07-01' AND '2023-02-28'"

    df = pd.read_sql_query(query, conn)

    # counting injuries per surface type
    grouped_data = df.groupby(['Surface', 'Injury']).size().reset_index(name='Count')

    # extract all the counts for the chart
    count_injured_hard = grouped_data[(grouped_data['Surface'] == 'Hard') & (grouped_data['Injury'] == 1)]['Count'].values[0] 
    count_non_injured_hard = grouped_data[(grouped_data['Surface'] == 'Hard') & (grouped_data['Injury'] == 0)]['Count'].values[0]

    count_injured_clay = grouped_data[(grouped_data['Surface'] == 'Clay') & (grouped_data['Injury'] == 1)]['Count'].values[0]
    count_non_injured_clay = grouped_data[(grouped_data['Surface'] == 'Clay') & (grouped_data['Injury'] == 0)]['Count'].values[0]

    count_injured_grass = grouped_data[(grouped_data['Surface'] == 'Grass') & (grouped_data['Injury'] == 1)]['Count'].values[0]
    count_non_injured_grass = grouped_data[(grouped_data['Surface'] == 'Grass') & (grouped_data['Injury'] == 0)]['Count'].values[0]

    # dump to JSON
    # wrapper for each variable in order to make it JSON serializable

    data = {
        "Hard": {
            "Injured": int(count_injured_hard),
            "Non-Injured": int(count_non_injured_hard)
        },
        "Clay": {
            "Injured": int(count_injured_clay),
            "Non-Injured": int(count_non_injured_clay)
        },
        "Grass": {
            "Injured": int(count_injured_grass),
            "Non-Injured": int(count_non_injured_grass)
        }
    }

    with open("injury_vs_court_type.json", "w") as f:
        json.dump(data, f)
        
generic_bar()        

In [ ]:
# close connection
conn.close()